Install Dependencies

In [1]:
!pip install -q transformers torch sentence-transformers faiss-cpu flask pyngrok accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 20.5 MB/s eta 0:00:00


Import libraries

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from flask import Flask, request, jsonify
from pyngrok import ngrok
import json
import threading
from typing import List, Dict, Any
import time

print("Libraries imported successfully!")

Libraries imported successfully!


Embedded Knowledge Base (18 Documents)

In [3]:
# @title
KNOWLEDGE_BASE = [
    {
        "id": "doc1",
        "title": "How to Make a Shoplite Account",
        "content": "Making a Shoplite account is pretty easy. You just go to the website and click Sign Up. You need to give your email address and make a password. The password should be at least 8 letters long and have one number in it. After you sign up, Shoplite sends you an email to check if it's really you. You have to click the link in the email within one day. Once you do that, your account is ready to use. You can shop right away if you're just buying things. If you want to sell things too, there's an extra step. You can also add your phone number later if you want extra security or to get text messages about your orders."
    },
    {
        "id": "doc2",
        "title": "Shopping Cart Basics",
        "content": "The shopping cart is where you put items you want to buy. You can add things from different sellers and they all stay in one cart. This is really convenient because you can buy multiple things at once. Your cart saves your items even if you close the browser and come back later. As long as you're logged in, your cart will remember everything for 30 days. After 30 days, if you haven't bought the items, they automatically get removed. You can also use discount codes in your cart. Just type the code in the box and it will take money off your total. If you're not ready to buy yet, you can save items to your wishlist instead."
    },
    {
        "id": "doc3",
        "title": "How to Check Out",
        "content": "Checking out means finishing your purchase. There are four main steps. First, you look at everything in your cart to make sure it's right. You can change quantities or remove items here. Second, you put in your shipping address. The website checks if your address is real while you type it. Third, you choose how to pay - credit card, PayPal, or Shoplite Wallet. Last, you confirm everything and place your order. Right after you pay, you'll get an email that says your order went through. This email has your order number and what you bought. It's good to save this email until you get your items."
    },
    {
        "id": "doc4",
        "title": "Payment Options and Safety",
        "content": "Shoplite takes most credit cards like Visa, MasterCard, and American Express. We also take PayPal and our own Shoplite Wallet. All of these ways are safe to use. We take security very seriously. When you pay with a card, we use something called 3D Secure which might ask for an extra code from your bank. We never save your full card number on our computers. Each time you buy something, you need to type the security code again even if you saved your card. If something looks strange about a purchase, we might ask you to confirm it's really you through email or phone."
    },
    {
        "id": "doc5",
        "title": "Tracking Your Order",
        "content": "After you buy something, you'll want to know where it is. Once the seller ships your item, you get a tracking number by email. You can also see it in your Shoplite account under My Orders. Regular shipping takes 3-5 business days. If you pay for express shipping, it comes in 1-2 days. For other countries, it might take 1-2 weeks. You'll get updates when your package leaves the warehouse, when it's out for delivery, and when it arrives. If no one is home when they try to deliver, they'll leave a note and keep the package at their local office for 5 days for you to pick up."
    },
    {
        "id": "doc6",
        "title": "Returns and Getting Your Money Back",
        "content": "If you don't like something you bought, you can usually return it within 30 days of when you got it. The item should be unused and in the original box with all the tags still on it. For things like phones and computers, you only have 14 days to return them. You have to pay to ship the item back to us unless it came broken or wrong. After we get your return and check that everything is okay, we give you your money back in 5-7 days. You can get your money back to your original payment method or as Shoplite credit. If you choose credit, we add 10% extra as a thank you."
    },
    {
        "id": "doc7",
        "title": "Writing Product Reviews",
        "content": "After you buy something, you can write a review about it. You have 90 days after buying to leave a review. You give 1 to 5 stars and write about what you liked or didn't like. Your review should be honest and about your actual experience with the product. We read all reviews before they go live to make sure they're appropriate. The seller can write a response to your review if they want to. Other people can vote if your review was helpful. If you write lots of good reviews, you might get special badges and see new products before other people."
    },
    {
        "id": "doc8",
        "title": "Becoming a Seller",
        "content": "If you want to sell on Shoplite, you need a seller account. This is different from a regular buyer account. You have to give us your business information and tax ID number so we know you're a real business. It takes us 2-3 days to check your information and approve your seller account. Once you're approved, you need to keep your customer ratings above 4 stars and answer customer questions within 2 days. Sellers get a special dashboard where they can see their sales, manage their products, and talk to customers."
    },
    {
        "id": "doc9",
        "title": "Managing Your Products as a Seller",
        "content": "As a seller, you can add, remove, or change your products in your dashboard. You can edit multiple products at once to save time. The website shows you how many of each item you have left. You can set it up so you get an alert when you're running low on something popular. This helps you never run out of stock. The inventory updates immediately when someone buys something so you don't accidentally sell things you don't have. If you use another system to manage inventory, you can connect it to Shoplite using our API."
    },
    {
        "id": "doc10",
        "title": "Seller Fees and Getting Paid",
        "content": "When you sell something on Shoplite, we take 8.5% of the sale price as our fee. There's also a 2.9% fee for payment processing. There's no monthly fee for a basic seller account. If you sell a lot, you might want a premium account that costs $29.99 per month but only charges 6.5% commission instead of 8.5%. We pay you every two weeks, but there's a 7-day waiting period after each sale first. You can see detailed reports about all your fees and sales in your seller dashboard."
    },
    {
        "id": "doc11",
        "title": "Getting Help from Customer Service",
        "content": "If you need help with anything, we have customer service available 24 hours a day, 7 days a week. You can chat with us live on the website (usually answers in 2 minutes), send us an email (we reply within 4 hours), or call us on the phone (we answer in under 5 minutes). If your problem is really complicated, it might take us 1-2 days to solve it completely. You can always check on your support tickets in your account. We also have FAQs and video guides that might answer your question faster."
    },
    {
        "id": "doc12",
        "title": "Shoplite Mobile App",
        "content": "Our mobile app has everything the website has plus some extra features. You can scan barcodes in stores to compare prices, use your camera to see how furniture might look in your room, and even search by shaking your phone. You can pay with Apple Pay or Google Pay for faster checkout. The app even works when you're offline - you can look at products you recently viewed or items in your saved cart without internet. You get special notifications on your phone about price drops and when your orders ship."
    },
    {
        "id": "doc13",
        "title": "Using Shoplite's API",
        "content": "If you're a developer, you can use our API to connect Shoplite with other programs. Our API lets you search products, manage orders, update inventory, and more (with permission). You can make up to 1000 requests per hour with one API key. You need to use OAuth 2.0 to prove who you are. Our API can also send automatic notifications to your system when things happen like new orders or inventory changes. We have examples in Python, JavaScript, and PHP to help you get started."
    },
    {
        "id": "doc14",
        "title": "Keeping Your Information Safe",
        "content": "We work hard to keep your information safe. We use SSL encryption (the same kind banks use) and regularly check our systems for security problems. Your personal data is stored in encrypted format. You can download all the information we have about you, ask us to delete it, or change your privacy settings in your account. We never sell your personal information to other companies. We follow privacy laws like GDPR and CCPA to make sure your rights are protected."
    },
    {
        "id": "doc15",
        "title": "Discount Codes and Sales",
        "content": "We have different kinds of discount codes. Some take a percentage off your total, some take off a fixed amount, some give you free shipping, and some let you buy one item and get another free. Most codes have a minimum purchase amount, and you usually can't use more than one code on the same order. We have big sales during holidays and special times of year. If you sign up for our emails, you'll get special codes that other people don't get and you'll find out about sales before everyone else."
    },
    {
        "id": "doc16",
        "title": "What Happens If Delivery Fails",
        "content": "Sometimes delivery doesn't work out. If the delivery person comes and no one is home, they'll leave a note and keep your package at their local office for 5 business days. You can go pick it up there with your ID. For expensive items over $500, someone might need to sign for the package. If there's any problem with delivery, you can report it right from your order tracking page."
    },
    {
        "id": "doc17",
        "title": "Saving Payment Methods",
        "content": "You can save your payment methods to make checkout faster. When you save a card, we don't save the security code, so you'll need to enter that each time you buy something. This is for your safety. All saved payment methods are encrypted and follow strict security standards. You can remove saved payment methods anytime in your account settings."
    },
    {
        "id": "doc18",
        "title": "Out of Stock Items in Cart",
        "content": "The cart shows you how many items are available in real time. If something sells out while it's in your cart, you'll see a message and the item will automatically disappear from your cart. Our system is designed to prevent overselling, so you'll never buy something that's actually out of stock. If you really want an item that sold out, you can sign up to get notified when it's back."
    }
]

print(f"Loaded {len(KNOWLEDGE_BASE)} Shoplite documents")

Loaded 18 Shoplite documents


Embedded Prompt Templates

In [4]:
# @title
PROMPT_TEMPLATES = {
    "base_retrieval": {
        "role": "You are Shoplite Assistant, a helpful customer service AI for Shoplite online shopping.",
        "goal": "Answer customer questions using only the Shoplite documents provided to you.",
        "guidelines": [
            "Only use information from the documents given to you",
            "If you don't have the information, say you don't know",
            "Be friendly and helpful",
            "Keep answers clear and simple",
            "Cite which documents you used"
        ],
        "format": "Answer: [Your response based on the documents]\nSources: [Which documents you used]"
    },
    "multi_document": {
        "role": "You are Shoplite Assistant, good at combining information from different documents.",
        "goal": "Answer complex questions that need information from multiple Shoplite documents.",
        "guidelines": [
            "Look at all the documents you are given",
            "Find which parts answer different parts of the question",
            "Put the information together in a logical way",
            "Don't repeat the same information multiple times",
            "Cite all documents you used"
        ],
        "format": "Answer: [Combined answer from multiple documents]\nSources: [List all documents used]"
    },
    "refusal": {
        "role": "You are Shoplite Assistant who knows when you can't answer a question.",
        "goal": "Politely say when you don't have the information to answer a question.",
        "guidelines": [
            "If no documents have the answer, say you can't help",
            "Suggest other topics you can help with",
            "Don't make up answers",
            "Stay friendly even when saying no"
        ],
        "format": "I'm sorry, but I don't have information about that in my Shoplite documents.\nI can help you with: [list 2-3 related topics you can help with]"
    },
    "clarification": {
        "role": "You are Shoplite Assistant who asks for clarification when questions are unclear.",
        "goal": "Get clearer questions so you can provide better answers.",
        "guidelines": [
            "If a question is confusing or could mean multiple things, ask for clarification",
            "Give examples of what would help you understand better",
            "Only ask for clarification when really needed"
        ],
        "format": "I want to make sure I give you the right information. Could you tell me more about:\n- [What specifically you need clarification on]\nFor example: [Give an example of what would help]"
    }
}

print("Prompt templates loaded")

Prompt templates loaded


Initialize Embedding Model and Create FAISS Index

In [5]:
# Initialize embedding model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings for all documents
document_texts = [doc["content"] for doc in KNOWLEDGE_BASE]
document_embeddings = embedding_model.encode(document_texts, normalize_embeddings=True)

# Create FAISS index
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # Using inner product for cosine similarity
index.add(document_embeddings.astype('float32'))

print(f"FAISS index created with {index.ntotal} documents")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FAISS index created with 18 documents


Initialize LLM

In [ ]:
# Cell 6: Initialize LLM
print("Loading Llama-3.1-8B model...")

try:
    from huggingface_hub import login

    print("Please enter your Hugging Face token to access Llama-3.1-8B")
    print("Get your token from: https://huggingface.co/settings/tokens")
    hf_token = input("Enter your Hugging Face token: ").strip()

    if hf_token:
        login(token=hf_token)
        print("Successfully authenticated with Hugging Face")

    model_name = "meta-llama/Llama-3.1-8B"
    print(f"Downloading {model_name}...")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    from transformers import BitsAndBytesConfig
    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        bnb_8bit_compute_dtype=torch.float16
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        quantization_config=quantization_config,
        trust_remote_code=True
    )

    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    print("Llama-3.1-8B loaded successfully!")

except Exception as e:
    print(f"Error loading Llama-3.1-8B: {e}")
    print("Please check your token and try again")

RAG Pipeline Functions

In [9]:
# Cell 7: RAG Pipeline Functions
print("Setting up RAG pipeline...")

def retrieve_documents(query: str, k: int = 3) -> List[Dict]:
    try:
        query_embedding = embedding_model.encode([query], normalize_embeddings=True)
        scores, indices = index.search(query_embedding.astype('float32'), k)

        relevant_docs = []
        for i, idx in enumerate(indices[0]):
            if idx < len(KNOWLEDGE_BASE):
                relevant_docs.append({
                    **KNOWLEDGE_BASE[idx],
                    "similarity_score": float(scores[0][i])
                })

        return relevant_docs
    except Exception as e:
        print(f"Error in retrieval: {e}")
        return []

def build_prompt(query: str, relevant_docs: List[Dict], prompt_type: str = "base_retrieval") -> str:
    template = PROMPT_TEMPLATES[prompt_type]

    context_parts = []
    for doc in relevant_docs:
        content = doc['content']
        if len(content) > 800:
            content = content[:800] + "..."
        context_parts.append(f"Document: {doc['title']}\nContent: {content}")

    context = "\n\n".join(context_parts)

    system_message = f"""<|start_header_id|>system<|end_header_id|>

{template['role']}
{template['goal']}

Guidelines:
{chr(10).join(['- ' + guideline for guideline in template['guidelines']])}

Context Documents:
{context}

Response Format:
{template['format']}<|eot_id|>"""

    user_message = f"""<|start_header_id|>user<|end_header_id|>

{query}<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>

"""

    return system_message + user_message

def generate_response(prompt: str, max_length: int = 512) -> str:
    try:
        outputs = pipe(
            prompt,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            top_k=50,
            repetition_penalty=1.1,
            return_full_text=False,
            eos_token_id=pipe.tokenizer.eos_token_id,
            pad_token_id=pipe.tokenizer.eos_token_id
        )

        response = outputs[0]['generated_text'].strip()

        if "<|eot_id|>" in response:
            response = response.split("<|eot_id|>")[0]

        return response

    except Exception as e:
        print(f"Generation error: {e}")
        return "I apologize, but I encountered an error while generating a response. Please try again."

Setting up RAG pipeline...


Flask API Setup

In [10]:
# Cell 8: Flask API Setup
print("Setting up Flask API...")

from flask import Flask, request, jsonify
import time

app = Flask(__name__)

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({
        "status": "healthy",
        "timestamp": time.time(),
        "documents_loaded": len(KNOWLEDGE_BASE)
    })

@app.route('/ping', methods=['POST'])
def ping():
    try:
        data = request.json
        message = data.get('message', 'Hello')

        prompt = f"<|start_header_id|>user<|end_header_id|>\n\n{message}<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n"
        response = generate_response(prompt)

        return jsonify({
            "response": response,
            "sources": [],
            "confidence": "direct_response"
        })

    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/chat', methods=['POST'])
def chat():
    try:
        data = request.json
        query = data.get('message', '')
        session_id = data.get('session_id', 'default')

        if not query:
            return jsonify({"error": "No message provided"}), 400

        print(f"Received query: {query}")

        relevant_docs = retrieve_documents(query, k=3)

        if not relevant_docs:
            response_text = "I'm sorry, but I don't have information about that in my Shoplite documents. I can help you with account creation, shopping, returns, seller information, and customer support."
            return jsonify({
                "response": response_text,
                "sources": [],
                "confidence": "low",
                "query": query
            })

        prompt_type = "multi_document" if len(relevant_docs) > 1 else "base_retrieval"
        prompt = build_prompt(query, relevant_docs, prompt_type)

        print("Generating response...")
        response_text = generate_response(prompt)

        sources = [doc['title'] for doc in relevant_docs]

        avg_similarity = sum(doc['similarity_score'] for doc in relevant_docs) / len(relevant_docs)
        confidence = "high" if avg_similarity > 0.5 else "medium" if avg_similarity > 0.3 else "low"

        return jsonify({
            "response": response_text,
            "sources": sources,
            "confidence": confidence,
            "query": query,
            "session_id": session_id
        })

    except Exception as e:
        print(f"Error in chat endpoint: {e}")
        return jsonify({
            "error": "Internal server error",
            "response": "I apologize, but I encountered an error. Please try again."
        }), 500

Setting up Flask API...


Start Flask Server with Ngrok

In [11]:
print("Starting server...")

def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

# Start Flask in a separate thread
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Wait for Flask to start
time.sleep(3)


Starting server...
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


Setup Ngrok Tunnel

In [ ]:
# Cell 10: Setup Ngrok Tunnel
print("Setting up ngrok tunnel...")

try:
    print("NGROK TOKEN REQUIRED")
    print("Get your free token from: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("Copy your authtoken and paste it below")

    ngrok_token = input("Enter your ngrok authtoken: ").strip()

    # ngrok token: "33X2K5WDaLQ7fMYjy4KdhCOLGIC_878yhT3jvuvbsn3xv13yf" (for sake of testing)

    if not ngrok_token:
        print("No ngrok token provided. Using free version (limited to 2 hours)")
        print("Free version may have connection limits")
    else:
        ngrok.set_auth_token(ngrok_token)
        print("Ngrok token set successfully")

    print("Creating secure tunnel...")
    public_url = ngrok.connect(5000, bind_tls=True)
    print(f"Your RAG API is now live at: {public_url}")
    print(f"Public URL: {public_url}")

    print("API ENDPOINTS:")
    print(f"   Health Check:   GET  {public_url}/health")
    print(f"   Direct LLM:     POST {public_url}/ping")
    print(f"   RAG Chat:       POST {public_url}/chat")

    print("USAGE:")
    print(f"   Use this URL in your chat_interface.py script")
    print(f"   Or test with: curl -X POST {public_url}/chat -H 'Content-Type: application/json' -d '{{\"message\": \"How do I create an account?\"}}'")

except Exception as e:
    print(f"Error setting up ngrok: {e}")
    print("Troubleshooting:")
    print("   - Make sure your ngrok token is correct")
    print("   - Check your internet connection")
    print("   - Try restarting the notebook if issues persist")

Setting up ngrok tunnel...
NGROK TOKEN REQUIRED
Get your free token from: https://dashboard.ngrok.com/get-started/your-authtoken
Copy your authtoken and paste it below
Enter your ngrok authtoken: 33X2K5WDaLQ7fMYjy4KdhCOLGIC_878yhT3jvuvbsn3xv13yf
Ngrok token set successfully
Creating secure tunnel...
Your RAG API is now live at: NgrokTunnel: "https://pseudomonocyclic-albina-unamatively.ngrok-free.dev" -> "http://localhost:5000"
Public URL: NgrokTunnel: "https://pseudomonocyclic-albina-unamatively.ngrok-free.dev" -> "http://localhost:5000"
API ENDPOINTS:
   Health Check:   GET  NgrokTunnel: "https://pseudomonocyclic-albina-unamatively.ngrok-free.dev" -> "http://localhost:5000"/health
   Direct LLM:     POST NgrokTunnel: "https://pseudomonocyclic-albina-unamatively.ngrok-free.dev" -> "http://localhost:5000"/ping
   RAG Chat:       POST NgrokTunnel: "https://pseudomonocyclic-albina-unamatively.ngrok-free.dev" -> "http://localhost:5000"/chat
USAGE:
   Use this URL in your chat_interface.py 

Test the System

In [ ]:
print("Testing the system...")

# Test health endpoint
import requests
try:
    health_response = requests.get("http://localhost:5000/health")
    print(f"Health check: {health_response.json()}")

    # Test a simple RAG query
    test_query = "How do I create a seller account?"
    test_response = requests.post("http://localhost:5000/chat", json={"message": test_query})

    if test_response.status_code == 200:
        result = test_response.json()
        print("Test query successful!")
        print(f"Question: {test_query}")
        print(f"Answer: {result['response'][:100]}...")
        print(f"Sources: {result['sources']}")
    else:
        print(f"Test query failed: {test_response.status_code}")

except Exception as e:
    print(f"Test failed: {e}")

print("Setup complete! Your RAG system is ready.")
print("Use the chat_interface.py script to interact with your API")
print("Or send POST requests to the /chat endpoint with JSON: {'message': 'your question'}")